## Question 1. Given two strings s1 and s2, return *the lowest **ASCII** sum of deleted characters to make two strings equal*.

In [1]:
def minimumDeleteSum(s1, s2, i, j, memo):
    # Base case: if either string is empty
    if i == len(s1):
        return sum(ord(ch) for ch in s2[j:])
    if j == len(s2):
        return sum(ord(ch) for ch in s1[i:])
    
    # Check if the solution has already been computed
    if memo[i][j] != -1:
        return memo[i][j]
    
    # If the characters match, no deletion needed
    if s1[i] == s2[j]:
        memo[i][j] = minimumDeleteSum(s1, s2, i+1, j+1, memo)
    else:
        # Find the minimum cost by deleting one character at a time
        delete_s1 = ord(s1[i]) + minimumDeleteSum(s1, s2, i+1, j, memo)
        delete_s2 = ord(s2[j]) + minimumDeleteSum(s1, s2, i, j+1, memo)
        memo[i][j] = min(delete_s1, delete_s2)
    
    return memo[i][j]

def minimumDeleteSumWrapper(s1, s2):
    # Initialize the memoization table with -1
    memo = [[-1] * len(s2) for _ in range(len(s1))]
    
    return minimumDeleteSum(s1, s2, 0, 0, memo)

## Question 2. Given a string s containing only three types of characters: '(', ')' and '*', return true *if* s *is **valid***.

### The following rules define a **valid** string:

- Any left parenthesis '(' must have a corresponding right parenthesis ')'.
- Any right parenthesis ')' must have a corresponding left parenthesis '('.
- Left parenthesis '(' must go before the corresponding right parenthesis ')'.
- '*' could be treated as a single right parenthesis ')' or a single left parenthesis '(' or an empty string "".

In [2]:
def isValid(s, i, count, memo):
    # Base case: if all characters have been processed
    if i == len(s):
        return count == 0

    # Check if the solution has already been computed
    if (i, count) in memo:
        return memo[(i, count)]

    # Get the current character
    ch = s[i]

    # Case 1: If the current character is '(', it must have a corresponding ')'
    if ch == '(':
        memo[(i, count)] = isValid(s, i + 1, count + 1, memo)
    # Case 2: If the current character is ')', it must have a corresponding '('
    elif ch == ')':
        if count == 0:
            memo[(i, count)] = False
        else:
            memo[(i, count)] = isValid(s, i + 1, count - 1, memo)
    # Case 3: If the current character is '*', it can be treated as '(', ')', or ''
    else:
        # Treat '*' as '('
        valid1 = isValid(s, i + 1, count + 1, memo)
        # Treat '*' as ')'
        valid2 = isValid(s, i + 1, count - 1, memo)
        # Treat '*' as ''
        valid3 = isValid(s, i + 1, count, memo)
        memo[(i, count)] = valid1 or valid2 or valid3

    return memo[(i, count)]

def checkValidString(s):
    memo = {}
    return isValid(s, 0, 0, memo)

## Question 3. Given two strings word1 and word2, return *the minimum number of **steps** required to make* word1 *and* word2 *the same*.

### In one **step**, you can delete exactly one character in either string.



In [3]:
def minDistance(word1, word2, i, j, memo):
    # Base cases: if either string is empty
    if i == len(word1):
        return len(word2) - j
    if j == len(word2):
        return len(word1) - i

    # Check if the solution has already been computed
    if memo[i][j] != -1:
        return memo[i][j]

    # If the characters match, move to the next characters
    if word1[i] == word2[j]:
        memo[i][j] = minDistance(word1, word2, i + 1, j + 1, memo)
    else:
        # Find the minimum steps by deleting one character from either string
        delete_word1 = 1 + minDistance(word1, word2, i + 1, j, memo)
        delete_word2 = 1 + minDistance(word1, word2, i, j + 1, memo)
        memo[i][j] = min(delete_word1, delete_word2)

    return memo[i][j]

def minDistanceWrapper(word1, word2):
    # Initialize the memoization table with -1
    memo = [[-1] * len(word2) for _ in range(len(word1))]

    return minDistance(word1, word2, 0, 0, memo)

## Question 4. You need to construct a binary tree from a string consisting of parenthesis and integers.

### The whole input represents a binary tree. It contains an integer followed by zero, one or two pairs of parenthesis. The integer represents the root's value and a pair of parenthesis contains a child binary tree with the same structure.

In [4]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def buildTreeFromString(s):
    def helper(s, index):
        if index >= len(s) or s[index] == ')':
            return None, index

        # Parse the integer value (can be multi-digit)
        num = ""
        while index < len(s) and s[index] not in ('(', ')'):
            num += s[index]
            index += 1

        root = TreeNode(int(num))
        
        # Check if there is a left child
        if index < len(s) and s[index] == '(':
            root.left, index = helper(s, index + 1)

        # Check if there is a right child
        if index < len(s) and s[index] == '(':
            root.right, index = helper(s, index + 1)

        # Move past the closing parenthesis
        if index < len(s) and s[index] == ')':
            index += 1

        return root, index

    root, _ = helper(s, 0)
    return root

# Helper function to print the binary tree in inorder traversal
def inorderTraversal(root):
    if root:
        inorderTraversal(root.left)
        print(root.val, end=' ')
        inorderTraversal(root.right)

## Question 5. Given an array of characters chars, compress it using the following algorithm:
## Begin with an empty string s. For each group of **consecutive repeating characters** in chars:

- If the group's length is 1, append the character to s.
- Otherwise, append the character followed by the group's length.

### The compressed string s **should not be returned separately**, but instead, be stored **in the input character array chars**. Note that group lengths that are 10 or longer will be split into multiple characters in chars.

### After you are done **modifying the input array,** return *the new length of the array*.

### You must write an algorithm that uses only constant extra space.

In [5]:
def compress(chars):
    def compressHelper(chars, start, count):
        if start >= len(chars):
            return count

        # Find the length of the current group
        end = start + 1
        while end < len(chars) and chars[end] == chars[start]:
            end += 1

        # Replace the group with the compressed characters
        if end - start > 1:
            chars[count] = chars[start]
            count += 1
            for digit in str(end - start):
                chars[count] = digit
                count += 1
        else:
            chars[count] = chars[start]
            count += 1

        # Recursively process the remaining characters
        return compressHelper(chars, end, count)

    return compressHelper(chars, 0, 0)

## Question 6. Given two strings s and p, return *an array of all the start indices of* p*'s anagrams in* s. You may return the answer in **any order**.

### An **Anagram** is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.

In [6]:
from collections import Counter

def findAnagrams(s, p):
    def isAnagram(s, p):
        return Counter(s) == Counter(p)

    def findAnagramsHelper(s, p, start, result):
        if start + len(p) > len(s):
            return

        if isAnagram(s[start:start+len(p)], p):
            result.append(start)

        findAnagramsHelper(s, p, start+1, result)

    result = []
    findAnagramsHelper(s, p, 0, result)
    return result

## Question 7. Given an encoded string, return its decoded string.

### The encoding rule is: k[encoded_string], where the encoded_string inside the square brackets is being repeated exactly k times. Note that k is guaranteed to be a positive integer.

### You may assume that the input string is always valid; there are no extra white spaces, square brackets are well-formed, etc. Furthermore, you may assume that the original data does not contain any digits and that digits are only for those repeat numbers, k. For example, there will not be input like 3a or 2[4].

### The test cases are generated so that the length of the output will never exceed 105.

In [1]:
def decodeString(s):
    def decodeStringHelper(s, i):
        decoded_string = ""
        num = 0

        while i < len(s):
            if s[i].isdigit():
                num = num * 10 + int(s[i])
                i += 1
            elif s[i] == "[":
                inner_string, i = decodeStringHelper(s, i + 1)
                decoded_string += num * inner_string
                num = 0
            elif s[i] == "]":
                return decoded_string, i + 1
            else:
                decoded_string += s[i]
                i += 1

        return decoded_string, i

    return decodeStringHelper(s, 0)[0]


## Question 8. Given two strings s and goal, return true *if you can swap two letters in* s *so the result is equal to* goal*, otherwise, return* false*.*

### Swapping letters is defined as taking two indices i and j (0-indexed) such that i != j and swapping the characters at s[i] and s[j].

In [2]:
def buddyStrings(s, goal):
    if len(s) != len(goal):
        return False

    if s == goal and len(set(s)) < len(s):
        return True

    diff_indices = []
    for i in range(len(s)):
        if s[i] != goal[i]:
            diff_indices.append(i)

    if len(diff_indices) != 2:
        return False

    i, j = diff_indices
    if s[i] == goal[j] and s[j] == goal[i]:
        return True

    return False